# Recipe QA

Installation of question answering module

In [2]:
!pip install flair
!pip install deeppavlov
!python -m deeppavlov install squad_bert
import nltk
nltk.download('punkt')

     |████████████████████████████████| 337kB 11.8MB/s 
     |████████████████████████████████| 798kB 34.4MB/s 
     |████████████████████████████████| 276kB 52.5MB/s 
     |████████████████████████████████| 19.7MB 161kB/s 
     |████████████████████████████████| 983kB 49.9MB/s 
     |████████████████████████████████| 71kB 12.4MB/s 
     |████████████████████████████████| 1.1MB 50.0MB/s 
     |████████████████████████████████| 1.3MB 54.9MB/s 
     |████████████████████████████████| 2.9MB 49.7MB/s 
     |████████████████████████████████| 890kB 45.8MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116677 sha256=677e4de23bebef4a6f0a973e61ff6bf7e49031d37787147149e62bdb78e99095
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for segtok: filename=segtok-1.5.10-cp36-none-any.whl size=25021 sha256=6565c9afa77290a80a9b91c4f63acaf768f516829f6e9d5ee749c10da98ba99c
  Stored in directory: /root/.cach

2020-10-25 01:57:39.717 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/squad/squad_bert.json'
     |████████████████████████████████| 110.5MB 114kB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 512kB 41.5MB/s 
     |████████████████████████████████| 3.8MB 46.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=9d9f0f0fed210129ff46ca8664be1d85fe2848f62772d4390aa71855c14f3271
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling ten

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from flair.models import SequenceTagger
from flair.data import Sentence
from nltk import tokenize
import pandas as pd
from tqdm import tqdm
import json
import re
import pprint

pos_tagger = SequenceTagger.load('pos')

!cd drive


2020-10-25 02:03:24,878 https://nlp.informatik.hu-berlin.de/resources/models/pos/en-pos-ontonotes-v0.5.pt not found in cache, downloading to /tmp/tmpkxtzsprv


100%|██████████| 249072763/249072763 [00:09<00:00, 25595599.74B/s]

2020-10-25 02:03:34,943 copying /tmp/tmpkxtzsprv to cache at /root/.flair/models/en-pos-ontonotes-v0.5.pt


2020-10-25 02:03:35,484 removing temp file /tmp/tmpkxtzsprv
2020-10-25 02:03:35,518 loading file /root/.flair/models/en-pos-ontonotes-v0.5.pt


Function to get answers for what based question
 

In [4]:
import pandas as pd
from flair.data import Sentence
from deeppavlov import build_model, configs
import re
search_pattern = re.compile(r"\bit\b")
from flair.models import SequenceTagger
model = build_model(configs.squad.squad, download=True)

pos_tagger = SequenceTagger.load('pos')

import spacy
import plac
import operator

nlp = spacy.load("en_core_web_sm")

def f(v):T,x,m='aeiou',"ed",v[-1];return[[[v+x,v+m+x][v[-2]in T and m and v[-3]not in T],[v+x,v[:-1]+"ied"][v[-2]not in T]][m=='y'],v+"d"][m=='e']

def get_answers_new(text):
  doc = nlp(text)
  root = []
  conj = []
  for word in doc:
    if word.dep_ == 'ROOT':
      root.append(word)
    elif word.dep_ == 'conj':
      conj.append(word)
  sentence = Sentence(text)
  pos_tagger.predict(sentence)
  nouns = {}
  for item in sentence.get_spans('pos'):
    if item.tag == 'NN' or item.tag == 'NNP' or item.tag == 'NNS':
      nouns.setdefault(item.text,0)

  answers = []
  for item in root:
    try:
      ques = "what is " + f(str(item)) +" ?"
      sol = model([text], [ques])[0][0]
      if sol in nouns.keys():
        nouns[sol] +=1
    except:
      print(text)

  for item in conj:
    try:
      ques = "what is " + f(str(item)) +" ?"
      sol = model([text], [ques])[0][0]
      if sol in nouns.keys():
        nouns[sol] +=1
    except:
      print(text)

  r = search_pattern.search(text)
  ques = "What is " + text[r.end():] +" ?"
  sol = model([text], [ques])[0][0]
  if sol in nouns.keys():
    nouns[sol] +=1

  ques = "What " + text[r.end():] +" ?"
  sol = model([text], [ques])[0][0]
  if sol in nouns.keys():
    nouns[sol] +=1

  if nouns == {}:
    max_vote = 'N/A'
  else:
    max_vote = max(nouns.items(), key=operator.itemgetter(1))[0]
    if nouns[max_vote] == 0:
      max_vote = 'N/A'
  
  return (max_vote,list(nouns.keys()))

2020-10-25 02:04:55.428 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/squad_model_1.4_cpu_compatible.tar.gz to /root/.deeppavlov/squad_model_1.4_cpu_compatible.tar.gz
100%|██████████| 222M/222M [00:44<00:00, 5.05MB/s]
2020-10-25 02:05:39.717 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /root/.deeppavlov/squad_model_1.4_cpu_compatible.tar.gz archive into /root/.deeppavlov/models
2020-10-25 02:05:44.87 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M.vec to /root/.deeppavlov/downloads/embeddings/wiki-news-300d-1M.vec
100%|██████████| 2.26G/2.26G [05:53<00:00, 6.39MB/s]
2020-10-25 02:11:38.374 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/embeddings/wiki-news-300d-1M-char.vec to /root/.deeppavlov/downloads/embeddings/wiki-news-300d-1M-char.vec
100%|█████████

2020-10-25 02:11:41.835 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 619: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead 

2020-10-25 02:11:43.213 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 619: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 
2020-10-25 02:11:43.325 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 619: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 
2020-10-25 02:11:43.400 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 619: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleGRUCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnGRUCell later. 


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-10-25 02:11:53.967 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/squad_model/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/squad_model/model
2020-10-25 02:11:54,678 loading file /root/.flair/models/en-pos-ontonotes-v0.5.pt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load the RecipeQA Dataset
json_data=open("/content/drive/My Drive/RecipeQA/train.json")
jdata = json.load(json_data)

import csv

# CSV File open
wikifile = open("/content/drive/My Drive/RecipeQA/recipeqa_it.csv",mode='a')
wikiwriter = csv.writer(wikifile, delimiter=',')

# Seperate the data from the JSON file
par = []
for obj in jdata['data']:
     for context in obj['context']:
          text = context['body'].lstrip()
          if text:
                text = re.sub(r'(?<=[.,!])(?=[^\s])', r' ', text)
                par.append(text)

#Remove parenthesis and other punctuation that does not add to the text
par_cleaned = []
for p in par:
    if '(' in p and ')' in p:
        p_cleaned = re.sub(r" ?\([^)]+\)", "", p)
    elif '[' in p and ']' in p:
        p_cleaned = re.sub(r" ?\[[^)]+\]", "", p)
    elif '{' in p and '}' in p:
        p_cleaned = re.sub(r" ?\{[^)]+\}", "", p)
    else:
        punc = "`~@#^&*<>=+_()[]{}|"
        p_cleaned = p
        for ch in p:
            if ch in punc:
                p_cleaned = p_cleaned.replace(ch, '')
    par_cleaned.append(p_cleaned)

raw_sentences = [tokenize.sent_tokenize(p) for p in par_cleaned]

# Check if the start  of the sentence is an alpha character
sent_final = []
for sent_list in raw_sentences:
  sent_list_temp = []
  for sent in sent_list:
    if sent[0].isalpha():
        sent_list_temp.append(sent)
  sent_final.append(sent_list_temp)

sent_final = sent_final[2251:2501]
print("Length of cleaned sentences: ",len(sent_final))

print("Tokenizing sentences for filter 2")
tokenized_sentences = []
for sentences_list in sent_final:
  sentences = [Sentence(s) for s in sentences_list]
  tokenized_sentences.append(sentences)

# POS tagging
idx = 0
print("Tagging sentences with POS")
for sentences_list in tokenized_sentences:
  for sentences in sentences_list:
    pos_tagger.predict(sentences)
  idx = idx+1
  print("POS tagging done for ",idx,".")
processed = []

# Only take sentences with "it" or "them" as the pronoun
print("\nApplying \'it\' filter...")
filter2_sentences = []
for sentences in tokenized_sentences:
  f = []
  prev_sent = []
  idx = 0
  for sent in tqdm(sentences):
    token_list = [token.text for token in sent]
    text = sent.to_original_text()
    pos_tag = [token.get_tag('pos').value for token in sent]
    if (token_list.count('it') == 1) and (pos_tag.count('PRP') + pos_tag.count('PRP$')) == 1:
        u = get_answers_new(text)
        if idx != 0 and u[0] == 'N/A' and ('NN' in pos_tag and pos_tag.index('NN') > pos_tag.index('PRP')):
          prev_sent[-1] = prev_sent[-1][:-1]
          text = text[:1].lower() + text[1:]
          text = prev_sent[-1]+", "+text # Add previous sentence for context
          u = get_answers_new(text)
        if u[0] != 'N/A' and len(u[1]) > 1:
            filter2_sentences.append([text,u[0],u[1]])
    prev_sent.append(text)
    idx = idx + 1
    

print("")
print("")
print("")

idx = 0
for i in filter2_sentences:
  wikiwriter.writerow(i)
  print("\"",i[0],"\",\"",i[1],"\",",i[2])
  if idx == 100:
    break
  idx = idx+1

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Length of cleaned sentences:  250
Tokenizing sentences for filter 2
Tagging sentences with POS
POS tagging done for  1 .
POS tagging done for  2 .
POS tagging done for  3 .
POS tagging done for  4 .
POS tagging done for  5 .
POS tagging done for  6 .
POS tagging done for  7 .
POS tagging done for  8 .
POS tagging done for  9 .
POS tagging done for  10 .
POS tagging done for  11 .
POS tagging done for  12 .
POS tagging done for  13 .
POS tagging done for  14 .
POS tagging done for  15 .
POS tagging done for  16 .
POS tagging done for  17 .
POS tagging done for  18 .
POS tagging done for  19 .
POS tagging done for  20 .
POS tagging done for  21 .
POS tagging done for  22 .
POS tagging done for  23 .
POS tagging done for  24 .
POS tagging done for  25 .
POS tagging done for  26 .
POS tagging done for  27 .
POS tagging done for  28 .
POS tagging done for  29 .
PO

In [ ]:
u = get_answers_new('Also I heard that you can skip putting in the lemon juice and let the soup set in your refrigerator for two or three days,This allows it to sourer on its own.')
print(u)
if(u[0] == 'N/A'):
  print("Hello")


lemon  :  0
juice  :  0
soup  :  0
refrigerator  :  1
days,This  :  0
('refrigerator', ['lemon', 'juice', 'soup', 'refrigerator', 'days,This'])
